In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import re

In [2]:
run = "all_new" # before_selection

## Overview

This is the entry point for the python code, this notebook 

- loads the feature vector
- sets column names
- formats timestamp into `DateTime` as sets as index
- exports to `feature_vec.csv`



## Feature Vector in

> Currently importing from a .csv file saved from the Play run, API code for play is at the bottom but not implemented yet

In [3]:
import glob

#l = [pd.read_csv(filename) for filename in glob.glob("../../../0-data/processed/*.txt")]
#df = pd.concat(l, axis=0)
#df

In [4]:
# Set the column names
feature_vector_keys = ["timestamp", "tweet_id", "positive_sentiment", "negative_sentiment", \
                       "numb_of_mentions", "numb_of_urls", "numb_of_media","numb_of_hashtags", \
                       "numb_of_personal_pronouns", "numb_of_present_tenses", "numb_of_past_tenses", \
                       "sent_from_web", "numb_of_weird_chars", "numb_of_questions", "numb_of_emoticons", \
                       "numb_of_swearing_words", "numb_of_slang_words", "numb_of_intensifiers", \
                       "tweet_length", "userFollowersCount","userFriendsCount", "user_numb_of_tweets",\
                       "user_list_count", "tfidf_fire", "dict_precision", "dict_recall", "dict_f_measure"
                       ]

fV = pd.read_csv("../../../0-data/processed/all_new.txt", sep=" ", header=None)

df = fV
df.columns = feature_vector_keys
df.info()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   timestamp                  501 non-null    object
 1   tweet_id                   501 non-null    object
 2   positive_sentiment         501 non-null    object
 3   negative_sentiment         501 non-null    object
 4   numb_of_mentions           501 non-null    object
 5   numb_of_urls               501 non-null    object
 6   numb_of_media              501 non-null    object
 7   numb_of_hashtags           501 non-null    object
 8   numb_of_personal_pronouns  501 non-null    object
 9   numb_of_present_tenses     501 non-null    object
 10  numb_of_past_tenses        501 non-null    object
 11  sent_from_web              501 non-null    object
 12  numb_of_weird_chars        501 non-null    object
 13  numb_of_questions          501 non-null    object
 14  numb_of_em

In [5]:
df= df.groupby(['tweet_id']).agg('first')
df.reset_index(level=0, inplace=True)
#df2.describe()

# Timestamp

Timestamp comes in as a float, can be converted to TimeStamp and set to index using this method

In [6]:
def trim(df):
    
    df['timestamp'] = df['timestamp'].str[1:]
    df['timestamp'] = df['timestamp'].str[:-1]
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    
def timestamp_format(df):
    # Convert the timestamp to a DateTime format
    df["timestamp"] = pd.to_numeric(df["timestamp"], errors='coerce')
    df['timestamp'] = pd.to_datetime(df['timestamp'],unit='ms')

    # Changes the decimal setting for float
    
    #pd.set_option('display.float_format', lambda x: '%.3f' % x)

    # Set the DateTime to the pandas index
    #df = df.set_index('timestamp')


trim(df)

#DateTime will break ML 
#timestamp_format(df)

df

,tweet_id,timestamp,positive_sentiment,negative_sentiment,numb_of_mentions,numb_of_urls,numb_of_media,numb_of_hashtags,numb_of_personal_pronouns,numb_of_present_tenses,...,numb_of_intensifiers,tweet_length,userFollowersCount,userFriendsCount,user_numb_of_tweets,user_list_count,tfidf_fire,dict_precision,dict_recall,dict_f_measure
0,"1.15780341411953869E18,",1.564876819E12,"2.0,","66.0,","2.0,","0.0,","0.0,","0.0,","0.0,","0.0,",...,"0.0,","68.0,","70.0,","140.0,","0.0,","1.0,","0.0,","0.0,","0.0,",0.0]
1,"1.15780463514873856E18,",1.56487711E12,"12.0,","48.0,","0.0,","1.0,","1.0,","0.0,","0.0,","0.0,",...,"0.0,","96.0,","1445.0,","4608.0,","0.0,","6.0,","0.0,","0.0,","0.0,",0.0]
2,"1.15780699436918784E18,",1.564877673E12,"1.0,","68.0,","1.0,","0.0,","0.0,","0.0,","0.0,","0.0,",...,"0.0,","51.0,","1288.0,","3510.0,","0.0,","33.0,","0.0,","0.0,","0.0,",0.0]
3,"1.15780702300371354E18,",1.564877679E12,"6.0,","63.0,","0.0,","0.0,","1.0,","0.0,","0.0,","0.0,",...,"0.0,","61.0,","1349.0,","1533.0,","0.0,","23.0,","0.0,","0.0,","0.0,",0.0]
4,"1.15781273568770048E18,",1.564879041E12,"27.0,","23.0,","1.0,","0.0,","0.0,","0.0,","0.0,","0.0,",...,"0.0,","67.0,","130.0,","148.0,","0.0,","2.0,","0.0,","0.0,","0.0,",0.0]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,"1.15888666073993216E18,",1.565135085E12,"12.0,","48.0,","0.0,","0.0,","0.0,","0.0,","0.0,","0.0,",...,"0.0,","272.0,","1.0,","1.0,","0.0,","0.0,","0.0,","0.0,","0.0,",0.0]
468,"1.15888841085104128E18,",1.565135502E12,"5.0,","60.0,","0.0,","0.0,","1.0,","0.0,","0.0,","0.0,",...,"0.0,","120.0,","92.0,","306.0,","0.0,","6.0,","0.0,","0.0,","0.0,",0.0]
469,"1.15888847134110515E18,",1.565135517E12,"3.0,","23.0,","0.0,","0.0,","1.0,","0.0,","0.0,","0.0,",...,"0.0,","131.0,","85.0,","2.0,","0.0,","2.0,","0.0,","0.0,","0.0,",0.0]
470,"1.15888868355207168E18,",1.565135567E12,"0.0,","1.0,","1.0,","0.0,","0.0,","0.0,","0.0,","0.0,",...,"0.0,","57.0,","642.0,","946.0,","0.0,","5.0,","0.0,","0.0,","0.0,",0.0]


In [7]:
#df = df.drop(['timestamp'],axis =1)

In [8]:
df['tweet_id'] = df['tweet_id'].str[:-1]
df['positive_sentiment'] = df['positive_sentiment'].str[:-1]
df['negative_sentiment'] = df['negative_sentiment'].str[:-1]
df['numb_of_questions'] = df['numb_of_questions'].str[:-1]
df['numb_of_mentions'] = df['numb_of_mentions'].str[:-1]
df['numb_of_urls'] = df['numb_of_urls'].str[:-1]
df['numb_of_media'] = df['numb_of_media'].str[:-1]
df['numb_of_personal_pronouns'] = df['numb_of_personal_pronouns'].str[:-1]
df['numb_of_present_tenses'] = df['numb_of_present_tenses'].str[:-1]
df['tfidf_fire'] = df['tfidf_fire'].str[:-1]
df['numb_of_intensifiers'] = df['numb_of_intensifiers'].str[:-1]
df['numb_of_hashtags'] = df['numb_of_hashtags'].str[:-1]

df['sent_from_web'] = df['sent_from_web'].str[:-1]
df['numb_of_weird_chars'] = df['numb_of_weird_chars'].str[:-1]
df['numb_of_questions'] = df['numb_of_questions'].str[:-1]
df['numb_of_swearing_words'] = df['numb_of_swearing_words'].str[:-1]
df['numb_of_emoticons'] = df['numb_of_emoticons'].str[:-1]
df['numb_of_slang_words'] = df['numb_of_slang_words'].str[:-1]
df['tweet_length'] = df['tweet_length'].str[:-1]
df['userFollowersCount'] = df['userFollowersCount'].str[:-1]
df['userFriendsCount'] = df['userFriendsCount'].str[:-1]
df['user_list_count'] = df['user_list_count'].str[:-1]
df['dict_precision'] = df['dict_precision'].str[:-1]
df['dict_recall'] = df['dict_recall'].str[:-1]
df['dict_f_measure'] = df['dict_f_measure'].str[:-1]

In [13]:
#df.tweet_id.map(lambda x: '{:.0f}'.format(x))
#df.tweet_id = df.tweet_id.map(lambda x: '{:.0f}'.format(x))
df['tweet_id'] = pd.to_numeric(df['tweet_id'].apply(lambda x: re.sub(',', '.', str(x))))
df['tweet_id'].describe()
df.head()

,tweet_id,timestamp,positive_sentiment,negative_sentiment,numb_of_mentions,numb_of_urls,numb_of_media,numb_of_hashtags,numb_of_personal_pronouns,numb_of_present_tenses,...,numb_of_intensifiers,tweet_length,userFollowersCount,userFriendsCount,user_numb_of_tweets,user_list_count,tfidf_fire,dict_precision,dict_recall,dict_f_measure
0,1157803414119538688.00,1564876819000.00,2.00,66.00,2.00,0.00,0.00,0.00,0.00,0.00,...,0.0,68.0,70.0,140.0,"0.0,",1.0,0.0,0.0,0.0,0.0
1,1157804635148738304.00,1564877110000.00,12.00,48.00,0.00,1.00,1.00,0.00,0.00,0.00,...,0.0,96.0,1445.0,4608.0,"0.0,",6.0,0.0,0.0,0.0,0.0
2,1157806994369187840.00,1564877673000.00,1.00,68.00,1.00,0.00,0.00,0.00,0.00,0.00,...,0.0,51.0,1288.0,3510.0,"0.0,",33.0,0.0,0.0,0.0,0.0
3,1157807023003713536.00,1564877679000.00,6.00,63.00,0.00,0.00,1.00,0.00,0.00,0.00,...,0.0,61.0,1349.0,1533.0,"0.0,",23.0,0.0,0.0,0.0,0.0
4,1157812735687700480.00,1564879041000.00,27.00,23.00,1.00,0.00,0.00,0.00,0.00,0.00,...,0.0,67.0,130.0,148.0,"0.0,",2.0,0.0,0.0,0.0,0.0


In [10]:
# Obj -> Str -> float
pd.set_option('display.float_format', '{:.2f}'.format)

for col in df.columns.values:
    print(col)
    df[col] = df[col].astype(str).astype(float)#.astype('int64')
    
float_col = df.select_dtypes(include=['float64'])



for col in float_col.columns.values:
    df[col] = df[col].astype('int64')



#df.astype(float)

display(df.dtypes) 

tweet_id
timestamp
positive_sentiment
negative_sentiment
numb_of_mentions
numb_of_urls
numb_of_media
numb_of_hashtags
numb_of_personal_pronouns
numb_of_present_tenses
numb_of_past_tenses


ValueError: could not convert string to float: '0.0,'

In [ ]:
df

# I/O

In [ ]:
def clean_dataset_int(df):
    #df = df_t.drop(['category','matchedName','eventType','eventID'],axis =1)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    #    df.round()
    df.replace(np.nan,0)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    df.astype(int)
    df = df.replace(r'\D+', '', regex=True)
    return df[indices_to_keep].astype(np.float64)

def export_df(df):
    # Remove Unnamed Columns
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    #df.astype(int)
    df = df.replace(r'\D+', '', regex=True)
    df.round()
    # Export
    df.to_csv("../3-csv/" + run + ".csv", index=False)

#pd.set_option('precision', 100)
#export_df(df)
df = clean_dataset_int(df)
df.to_csv("../3-csv/" + run + ".csv", index=False)

In [ ]:
def import_df():
    # Import
    df = pd.read_csv(run + ".csv")
    # Timestamp will need to be reconfigured as index on each load
    #df = df.set_index('timestamp')
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    df

import_df()

df

## Play API

In [ ]:
## API
'''
import http.client

conn = http.client.HTTPConnection("localhost:9000")
conn.request("GET", "/vec")
res = conn.getresponse()
print(res.status, res.reason)

conn.close()
'''